<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](./data/mountain/art1131.jpg)|![](./data/forest/cdmc317.jpg)|

The problem is realively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be something that you can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several differnet possible models.

# Pre - Trained Model
<a id="p1"></a>

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D()
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

The `include_top` parameter in `ResNet50` will remove the full connected layers from the ResNet model. The next step is to turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes. 

```python
for layer in resnet.layers:
    layer.trainable = False
```

Using the Keras functional API, we will need to additional full connected layers to our model. We removed the top layers, we removed all previous fully connected layers. In other words, we kept only the feature processing portions of our network. You can expert with additional layers beyond what's listed here. The `GlobalAveragePooling2D` layer functions as a really fancy flatten function by taking the average of each of the last convolutional layer outputs (which is two dimensional still). 

```python
x = res.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(res.input, predictions)
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pretrained layers from resnet
4. Report your model's accuracy

## Load in Data

![skimage-logo](https://scikit-image.org/_static/img/logo.png)

Check out out [`skimage`](https://scikit-image.org/) for useful functions related to processing the images. In particular checkout the documentation for `skimage.io.imread_collection` and `skimage.transform.resize`.

In [9]:
# imports.
import numpy as np
from numpy import asarray
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model # This is the functional API
from skimage.transform import resize
from skimage import color, io
from skimage.exposure import rescale_intensity

In [10]:
# set the mountain columns.
col_mtn = io.ImageCollection('../data/mountain/*')
# set the forest columns.
col_fst = io.ImageCollection('../data/forest/*')

In [11]:
# show the shape.
print(col_mtn[0].shape)
print(col_fst[0].shape)

(256, 256, 3)
(256, 256, 3)


In [12]:
# show the length.
len(col_mtn)

374

In [13]:
# create image process function.
def image_process(coll1, coll2):
    i_list = []
    d_list = []
    l_list = []
    
    for i in range(0, len(coll1)):
                
        image = coll1[i]
        data = asarray(image)
                
        # load images into image_list
        i_list.append(data)
  
        # load images as ravelled array into data_list
        rav_array = np.ravel(data)
        d_list.append(rav_array)

        l_list.append(1)
        
    for i in range(0, len(coll2)):
                
        image = coll2[i]
        data = asarray(image)
                
        # load images into image_list
        i_list.append(data)
  
        # load images as ravelled array into data_list
        rav_array = np.ravel(data)
        d_list.append(rav_array)

        l_list.append(0)
                
    # convert image_list to numpy array
    i_list = np.array(i_list)

    # convert data_list to numpy array
    d_list = np.array(d_list)

    # convert label_list to numpy array
    l_list = np.array(l_list)
        
    return i_list, d_list, l_list

image_list, data_list, label_list = image_process(col_mtn, col_fst)

print(image_list.shape)
print(data_list.shape)
print(label_list.shape)

(702, 256, 256, 3)
(702, 196608)
(702,)


In [19]:
# set the x and y values.
X = image_list
y = label_list

# split into a training and testing set.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## Instatiate Model

In [15]:
# set the resnet data.
resnet = ResNet50(weights='imagenet', include_top=False)
for layer in resnet.layers:
    layer.trainable = False

## Fit Model

In [17]:
# create the model.
x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)
# complie the model.
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy'])
# fit the model with parameters.
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Train on 526 samples, validate on 176 samples
Epoch 1/5
526/526 [==============================] - 81s 153ms/sample - loss: 0.1156 - accuracy: 0.8498 - val_loss: 0.0991 - val_accuracy: 0.8807
Epoch 2/5
526/526 [==============================] - 84s 160ms/sample - loss: 0.0294 - accuracy: 0.9658 - val_loss: 0.0527 - val_accuracy: 0.9375
Epoch 3/5
526/526 [==============================] - 86s 163ms/sample - loss: 0.0234 - accuracy: 0.9753 - val_loss: 0.0709 - val_accuracy: 0.9205
Epoch 4/5
526/526 [==============================] - 76s 144ms/sample - loss: 0.0156 - accuracy: 0.9829 - val_loss: 0.1046 - val_accuracy: 0.8920
Epoch 5/5
526/526 [==============================] - 75s 143ms/sample - loss: 0.0277 - accuracy: 0.9696 - val_loss: 0.0348 - val_accuracy: 0.9545


# Custom CNN Model

In this step, write and train your own convolutional neural network using Keras. You can use any architecture that suits you as long as it has at least one convolutional and one pooling layer at the beginning of the network - you can add more if you want. 

In [21]:
# imports.
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

In [22]:
# create image process function.
def image_process(coll1, coll2):
    i_list = []
    d_list = []
    l_list = []
    
    for i in range(0, len(coll1)):
                
        image = resize(coll1[i], (coll1[i].shape[0] // 8, coll1[i].shape[1] // 8),
                       anti_aliasing=True)
        data = asarray(image)
                
        # load images into image_list
        i_list.append(data)
  
        # load images as ravelled array into data_list
        rav_array = np.ravel(data)
        d_list.append(rav_array)

        l_list.append(1)
        
    for i in range(0, len(coll2)):
                
        image = image = resize(coll2[i], (coll2[i].shape[0] // 8, coll2[i].shape[1] // 8),
                       anti_aliasing=True)
        data = asarray(image)
                
        # load images into image_list
        i_list.append(data)
  
        # load images as ravelled array into data_list
        rav_array = np.ravel(data)
        d_list.append(rav_array)

        l_list.append(0)
                
    # convert image_list to numpy array
    i_list = np.array(i_list)

    # convert data_list to numpy array
    d_list = np.array(d_list)

    # convert label_list to numpy array
    l_list = np.array(l_list)
        
    return i_list, d_list, l_list

image_list, data_list, label_list = image_process(col_mtn, col_fst)

print(image_list.shape)
print(data_list.shape)
print(label_list.shape)

(702, 32, 32, 3)
(702, 3072)
(702,)


In [23]:
# set the x and y values.
X = image_list
y = label_list

# split into a training and testing set.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [24]:
# create the model.
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))
# show the model summary.
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                6

In [25]:
# compile the model.
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [26]:
# fit the model.
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

Train on 526 samples, validate on 176 samples
Epoch 1/20
526/526 [==============================] - 1s 2ms/sample - loss: 0.9563 - accuracy: 0.4848 - val_loss: 0.6328 - val_accuracy: 0.5966
Epoch 2/20
526/526 [==============================] - 0s 799us/sample - loss: 0.6448 - accuracy: 0.5913 - val_loss: 0.5361 - val_accuracy: 0.8295
Epoch 3/20
526/526 [==============================] - 0s 826us/sample - loss: 0.4324 - accuracy: 0.8384 - val_loss: 0.4300 - val_accuracy: 0.7898
Epoch 4/20
526/526 [==============================] - 0s 806us/sample - loss: 0.4176 - accuracy: 0.8289 - val_loss: 0.5058 - val_accuracy: 0.7216
Epoch 5/20
526/526 [==============================] - 0s 779us/sample - loss: 0.4308 - accuracy: 0.8308 - val_loss: 0.3455 - val_accuracy: 0.8750
Epoch 6/20
526/526 [==============================] - 1s 1ms/sample - loss: 0.3200 - accuracy: 0.8802 - val_loss: 0.3418 - val_accuracy: 0.8636
Epoch 7/20
526/526 [==============================] - 1s 1ms/sample - loss: 0.2864

# Custom CNN Model with Image Manipulations
## *This a stretch goal, and it's relatively difficult*

To simulate an increase in a sample of image, you can apply image manipulation techniques: cropping, rotation, stretching, etc. Luckily Keras has some handy functions for us to apply these techniques to our mountain and forest example. Check out these resources to help you get started: 

1. [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class)
2. [Building a powerful image classifier with very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
 

In [ ]:
# State Code for Image Manipulation Here

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language